Import Libraries

In [15]:
import os
import re

from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

OS

In [16]:
folder_path = 'LA-Audios' #SET FOLDER PATH
files = os.listdir(folder_path)

Data Pre

In [17]:
text_blobs_master_ls = []

def parse_speaker_text(text):
    pattern = r'\[(\d{2}:\d{2}:\d{2}\.\d{3})\] - (Speaker \d+)\n(.*?)\n(?=\[|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    text_blobs = [match[2].strip() for match in matches]

    return text_blobs

for file in files:
  with open(folder_path + "/" + file, 'r') as file:
    file_text = file.read()
    text_blobs = parse_speaker_text(file_text)

    filtered_text_blobs = []
    #remove short blobs (less 50)
    for blob in text_blobs:
      if len(blob) >= 50:
        filtered_text_blobs.append(blob)

    text_blobs_master_ls.extend(filtered_text_blobs)

len(text_blobs_master_ls)

4030

Inference

In [26]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"


tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

scores_ls = []
for text in text_blobs_master_ls:
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_ls.append(scores)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
scores_ls

array([0.93397576, 0.05625462, 0.00976964], dtype=float32)